In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re
import lightgbm as lgb
import datetime as dt
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [2]:
from sklearn.impute import SimpleImputer
from sklearn import svm

In [3]:
borrar = ["Opportunity_Name", "ID", "Brand", "Product_Type", "ASP_converted_Currency", 
          "Prod_Category_A", "Product_Category_B", "Actual_Delivery_Date", "Last_Activity", 
          "ASP_converted_Currency", "Prod_Category_A", "Product_Name", "Delivery_Year", "Month", 
          "TRF", "Product_Family", "Account_Name"]

useless = ["Submitted_for_Approval", "Account_Type", "Delivery_Terms", "Size", "Price", "ASP_Currency", 
           "Total_Amount_Currency", "Total_Taxable_Amount_Currency","Quote_Type", "Opportunity_Type"] # "Product_Category_B", "Region"]

dates = ["Account_Created_Date", "Opportunity_Created_Date", "Quote_Expiry_Date", "Last_Modified_Date", 
         "Planned_Delivery_Start_Date", "Planned_Delivery_End_Date"]

target = ["Opportunity_ID", "Stage", "Sales_Contract_No"]

In [4]:
def preprocess(data):
    
    # elimino caracteres prohibidos en los headers
    
    data = data.rename(columns = lambda x:re.sub("[^A-Za-z0-9_]+", "", x))
        
    for d in dates:
        data[d] = pd.to_datetime(data[d])

    # agrego features
    
    data["Contacts"] = data.groupby("Opportunity_ID", sort = False)["Opportunity_ID"].transform("count")
    data["Delivery_Difference"] = (data["Planned_Delivery_End_Date"] - data["Planned_Delivery_Start_Date"]).dt.days
    data["Same_Owner"] = (data.Opportunity_Owner == data.Account_Owner) & (data.Opportunity_Owner == data.Last_Modified_By)
    data["Has_Brand"] = data.Brand != "None"
    data["Has_Contract"] = data.Sales_Contract_No != "None"
    data["Different_Country"] = (data.Billing_Country != data.Territory) & (data.Territory != "None")    
    data.loc[data.TRF == 0, "TRF_Cat"] = 0
    data.loc[(1 <= data.TRF) & (data.TRF <= 7), "TRF_Cat"] = 1
    data.loc[data.TRF > 7, "TRF_Cat"] = 2
    data["Sales"] = data.groupby("Account_Name", sort = False)["Account_Name"].transform("count")
    
    data["Concrete_Offer"] = (data["Planned_Delivery_End_Date"] - data["Opportunity_Created_Date"]).dt.days
    
    data["Offer_Duration"] = (data["Quote_Expiry_Date"] - data["Opportunity_Created_Date"]).dt.days
    
    # fabri
    
    data["Territory_Defined"] = data.Territory != "None"
    data["Past_Quote"] = (data["Last_Modified_Date"] - data["Quote_Expiry_Date"]).dt.days
    
    # casteo a categoricas varias columnas
    categorical = [x for x in data.columns if data[x].dtype == "object"]
    for c in categorical:       
        data[c] = data[c].astype('category')
        
    # limpio columnas
    
    data = data.drop(borrar + useless + dates, axis = 1)
    
    #data.drop(data[(data.Contacts > 1) & (data.ASP.isna())].index, inplace = True)
    #data = data.drop_duplicates([x for x in data.columns if x != 'ID'])
    
    return (data)

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

In [6]:
data = pd.read_csv("Entrenamieto_ECI_2020.csv")

In [7]:
rest = [x for x in data.columns if x != 'ID']
len(data[data.duplicated(rest)])/float(len(data)) * 100

2.643535729037588

In [8]:
prep = preprocess(data)

In [9]:
prep = prep[(prep.Stage == "Closed Won") | (prep.Stage == "Closed Lost")]
prep.Stage = prep.Stage.replace({"Closed Won": 1, "Closed Lost": 0})

In [10]:
prep.isna().any()

Region                              False
Territory                           False
PricingDelivery_Terms_Quote_Appr    False
PricingDelivery_Terms_Approved      False
Bureaucratic_Code_0_Approval        False
Bureaucratic_Code_0_Approved        False
Bureaucratic_Code                   False
Source                              False
Billing_Country                     False
Opportunity_ID                      False
Sales_Contract_No                   False
Account_Owner                       False
Opportunity_Owner                   False
Currency                            False
Last_Modified_By                    False
ASP                                  True
ASP_converted                        True
Delivery_Quarter                    False
Total_Amount                         True
Total_Taxable_Amount                False
Stage                               False
Contacts                            False
Delivery_Difference                  True
Same_Owner                        

In [11]:
prep.drop(["Past_Quote", "Offer_Duration", "Concrete_Offer"], axis = 1, inplace = True)

In [12]:
cat_vars = [x for x in prep.select_dtypes('category').columns if x != 'Stage']

In [13]:
label_enc = LabelEncoder()

In [14]:
for col in cat_vars:
    prep[col] = label_enc.fit_transform(prep[col])

In [15]:
imputer = SimpleImputer(strategy='mean')

In [16]:
cols = prep.columns

In [17]:
cols

Index([u'Region', u'Territory', u'PricingDelivery_Terms_Quote_Appr', u'PricingDelivery_Terms_Approved', u'Bureaucratic_Code_0_Approval', u'Bureaucratic_Code_0_Approved', u'Bureaucratic_Code', u'Source', u'Billing_Country', u'Opportunity_ID', u'Sales_Contract_No', u'Account_Owner', u'Opportunity_Owner', u'Currency', u'Last_Modified_By', u'ASP', u'ASP_converted', u'Delivery_Quarter', u'Total_Amount', u'Total_Taxable_Amount', u'Stage', u'Contacts', u'Delivery_Difference', u'Same_Owner', u'Has_Brand', u'Has_Contract', u'Different_Country', u'TRF_Cat', u'Sales', u'Territory_Defined'], dtype='object')

In [18]:
prep = imputer.fit_transform(prep)

In [19]:
prep

array([[ 2., 46.,  1., ...,  2., 63.,  0.],
       [ 2., 46.,  0., ...,  0., 63.,  0.],
       [ 1., 43.,  0., ...,  0., 91.,  1.],
       ...,
       [ 2.,  3.,  1., ...,  0.,  3.,  1.],
       [ 1., 42.,  1., ...,  1.,  2.,  1.],
       [ 1., 42.,  1., ...,  2.,  9.,  1.]])

In [20]:
prep = pd.DataFrame(prep, columns = cols)

In [21]:
x_train, x_test, y_train, y_test = train_test_split(prep, prep.Stage, test_size = 0.3, random_state = 0)

In [22]:
model = svm.SVC(random_state = 42, probability = True, gamma = 'scale', kernel = 'poly', degree = 3)

In [23]:
model.fit(X = x_train.drop(target, axis = 1), y = y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
  max_iter=-1, probability=True, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [24]:
y_pred = model.predict_proba(x_test.drop(target, axis = 1))

In [26]:
y_pred

array([[0.42844157, 0.57155843],
       [0.42844158, 0.57155842],
       [0.42844158, 0.57155842],
       ...,
       [0.42844158, 0.57155842],
       [0.42844155, 0.57155845],
       [0.42844159, 0.57155841]])

In [38]:
for x in res[:10]:
    print("{:f}".format(x))

0.652177
0.658534
0.653449
0.657376
0.629896
0.655262
0.609040
0.646850
0.623969
0.653479


In [36]:
x_test.Stage.head(10)

2040     0.0
4069     0.0
4230     1.0
578      1.0
12594    0.0
6042     1.0
2763     0.0
15900    0.0
2669     1.0
12802    0.0
Name: Stage, dtype: float64

In [39]:
fuck = [x for x in res if x >= 0.025 and x <= 0.95]
len(fuck)

5065

In [40]:
len(fuck)/float(len(res)) * 100

100.0

In [41]:
pred = pd.DataFrame(res, index = x_test.index, columns = ["Prediction"])

In [42]:
x_test_pred = x_test.join(pred)
answer = x_test_pred.groupby("Opportunity_ID")["Stage","Prediction"].mean()

In [43]:
log_loss(answer["Stage"], answer["Prediction"])

0.64503430109244

In [26]:
predictors = [x for x in x_train.drop(target, axis = 1).columns]
feat_imp = pd.Series(model.feature_importances_, predictors).sort_values(ascending = False)
feat_imp = feat_imp[0:28]
plt.rcParams['figure.figsize'] = 28, 5
feat_imp.plot(kind = 'bar', title = 'Feature Importance')

In [27]:
bad = x_test_pred[(x_test_pred.Prediction > 0.5) & (x_test_pred.Stage == 0) | (x_test_pred.Prediction < 0.5) & (x_test_pred.Stage == 1)]

In [28]:
bad

,Region,Territory,PricingDelivery_Terms_Quote_Appr,PricingDelivery_Terms_Approved,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Source,Billing_Country,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Currency,Last_Modified_By,ASP,ASP_converted,Delivery_Quarter,Total_Amount,Total_Taxable_Amount,Stage,Contacts,Delivery_Difference,Same_Owner,Has_Brand,Has_Contract,Different_Country,TRF_Cat,Sales,Concrete_Offer,Offer_Duration,Territory_Defined,Past_Quote,Prediction
7812,APAC,Australia,1,0,1,0,Bureaucratic_Code_5,None,Australia,4832,2252,Person_Name_43,Person_Name_19,None,Person_Name_47,0.5400,0.38211,Q4,109620.00,109620.00,0,1,7.0,False,False,True,False,0.0,317,181.0,130.0,True,131.0,0.969321
4773,APAC,India,1,1,1,1,Bureaucratic_Code_4,Source_13,Sri Lanka,2824,904,Person_Name_49,Person_Name_49,None,Person_Name_47,0.3600,0.36000,Q4,157172.40,157172.40,0,1,19.0,False,False,True,True,0.0,31,72.0,72.0,True,-27.0,0.996730
9545,EMEA,Germany,0,0,0,0,Bureaucratic_Code_4,Source_9,Germany,6045,None,Person_Name_13,Person_Name_13,None,Person_Name_47,0.4200,0.47506,Q3,30450.00,82700.00,1,2,4.0,False,False,False,False,0.0,125,32.0,4.0,True,160.0,0.365609
9800,APAC,Thailand,0,0,0,0,Bureaucratic_Code_4,Source_7,Thailand,6207,3436,Person_Name_54,Person_Name_54,None,Person_Name_47,0.4000,0.40000,Q4,120060.00,120060.00,0,1,0.0,False,False,True,False,0.0,1,86.0,57.0,True,70.0,0.977528
952,EMEA,Germany,1,0,0,0,Bureaucratic_Code_5,Source_7,Germany,511,1344,Person_Name_4,Person_Name_4,None,Person_Name_41,0.3875,0.43830,Q2,287525.00,287525.00,0,1,12.0,False,False,True,False,1.0,17,88.0,8.0,True,603.0,0.993678
16800,EMEA,None,0,0,0,0,Bureaucratic_Code_4,None,Luxembourg,12726,None,Person_Name_18,Person_Name_20,None,Person_Name_20,0.5400,0.61079,Q1,105300.00,105300.00,1,1,3.0,False,False,False,False,0.0,2,86.0,72.0,False,-8.0,0.113047
9280,Americas,NW America,0,0,0,0,Bureaucratic_Code_4,None,United States,5834,2880,Person_Name_64,Person_Name_64,None,Person_Name_47,0.4300,0.43000,Q4,30702.00,30702.00,0,1,0.0,False,False,True,True,0.0,22,132.0,4.0,True,183.0,0.630035
10407,Americas,None,1,0,0,0,Bureaucratic_Code_5,Source_7,United States,6515,3390,Person_Name_33,Person_Name_33,None,Person_Name_47,0.4950,0.49500,Q1,285862.50,285862.50,0,1,30.0,False,False,True,False,1.0,73,89.0,NaN,False,NaN,0.978764
15533,EMEA,Germany,1,1,1,1,Bureaucratic_Code_4,Source_7,Germany,9934,None,Person_Name_62,Person_Name_62,None,Person_Name_62,0.2300,0.26015,Q4,10666.25,10666.25,1,1,2.0,True,False,False,False,0.0,39,6.0,NaN,True,NaN,0.180077
7806,APAC,Australia,1,1,1,1,Bureaucratic_Code_4,None,Australia,4826,2247,Person_Name_43,Person_Name_19,None,Person_Name_47,0.5400,0.38211,Q4,109620.00,109620.00,0,1,7.0,False,False,True,False,0.0,317,181.0,130.0,True,131.0,0.813893


In [29]:
bad.Stage.value_counts()

0    78
1    21
Name: Stage, dtype: int64

In [30]:
validation_file = "Validacion_ECI_2020.csv"
vali = pd.read_csv(validation_file)
validation = preprocess(vali)
leak = ["Opportunity_ID", "Sales_Contract_No"]
pred = model.predict_proba(validation.drop(leak, axis = 1))[:,1]

# agrupo por Opportunity_ID para dar una sola prediccion por solicitud

pred = pd.DataFrame(pred, index = validation.index, columns = ["Prediction"])
validation = validation.join(pred)

answer = pd.DataFrame(validation.groupby("Opportunity_ID", as_index = False)["Prediction"].mean())

In [31]:
prev = pd.read_csv("acceptable/submission_23.csv", names=["Opportunity_ID", "Prediction"])
prev["Prediction"].corr(answer["Prediction"])

0.9966014072053296

In [32]:
prev = pd.read_csv("acceptable/submission_11.csv")
prev["Prediction"].corr(answer["Prediction"])

0.993107873090827